# Traffic Sign Indentification

In [ ]:
!pip install -r setup_lib.txt

In [2]:
import os
import pandas as pd
from imageio import imread
import math
import numpy as np
import cv2
import tensorflow
import keras
from tensorflow.keras.layers import Dense, Dropout, Flatten, Input
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential

In [ ]:
data_path = 'GTSRB/Final_Training/Images'
pixels = []
labels = []
# Loop qua các thư mục trong thư mục Images
for dir in os.listdir(data_path):
    # Bỏ qua file .DS_Store của máy MacOS
    if dir == '.DS_Store':
        continue

    # Đọc file csv để lấy thông tin về ảnh
    class_dir = os.path.join(data_path, dir)
    info_file = pd.read_csv(os.path.join(class_dir, "GT-" + dir + '.csv'), sep=';')

    # Lăp trong file
    for row in info_file.iterrows():
        # Đọc ảnh
        pixel = imread(os.path.join(class_dir, row[1].Filename))
        # Trích phần ROI theo thông tin trong file csv
        pixel = pixel[row[1]['Roi.X1']:row[1]['Roi.X2'], row[1]['Roi.Y1']:row[1]['Roi.Y2'], :]
        # Resize về kích cỡ chuẩn
        img = cv2.resize(pixel, (64,64))

        # Thêm vào list dữ liệu
        pixels.append(img)

        # Thêm nhãn cho ảnh
        labels.append(row[1].ClassId)
print("Finish")

Trộn dữ liệu và chia dữ liệu thành 3 phần với tỉ lệ Train:Validation:Test = 6:2:2

In [ ]:
# Chuẩn hoá dữ liệu pixels và labels
pixels = np.array(pixels)
labels = keras.utils.np_utils.to_categorical(labels)

# Nhào trộn dữ liệu ngẫu nhiên
randomize = np.arange(len(pixels))
np.random.shuffle(randomize)
X = pixels[randomize]
y = labels[randomize]

print("X = ",X.shape)

# Chia dữ liệu theo tỷ lệ 60% train và 40% còn lại cho val và test
train_size = int(X.shape[0] * 0.6)
X_train, X_val_test = X[:train_size], X[train_size:]
y_train, y_val_test = y[:train_size], y[train_size:]

val_size = int(X_val_test.shape[0] * 0.5) # 50% của phần 40% bên trên
X_val, X_test = X_val_test[:val_size], X_val_test[val_size:]
y_val, y_test = y_val_test[:val_size], y_val_test[val_size:]

print("X_train = ",X_train.shape)
print("X_val = ",X_val.shape)
print("X_test = ",X_test.shape)

Xây dựng model sử dụng Transfer Learning với mạng VGG16

In [ ]:
input_shape=(64,64,3)
filter_size = (3,3)
pool_size = (2, 2)
output_size = 43
model = Sequential()

model.add(Conv2D(16, filter_size, activation="relu", padding="same", input_shape=(64, 64, 3)))
model.add(BatchNormalization())
model.add(Conv2D(16, filter_size, activation="relu", padding="same"))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=pool_size, strides=(2,2)))

model.add(Conv2D(32, filter_size, activation="relu", padding="same"))
model.add(BatchNormalization())
model.add(Conv2D(32, filter_size, activation="relu", padding="same"))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=pool_size, strides=(2,2)))

model.add(Conv2D(64, filter_size, activation="relu", padding="same"))
model.add(BatchNormalization())
model.add(Conv2D(64, filter_size, activation="relu", padding="same"))
model.add(BatchNormalization())
model.add(Conv2D(64, filter_size, activation="relu", padding="same"))
model.add(MaxPooling2D(pool_size=pool_size, strides=(2,2)))

model.add(Conv2D(128, filter_size, activation="relu", padding="same"))
model.add(BatchNormalization())
model.add(Conv2D(128, filter_size, activation="relu", padding="same"))
model.add(BatchNormalization())
model.add(Conv2D(128, filter_size, activation="relu", padding="same"))
model.add(MaxPooling2D(pool_size=pool_size, strides=(2,2)))

model.add(Conv2D(256, filter_size, activation="relu", padding="same"))
model.add(BatchNormalization())
model.add(Conv2D(256, filter_size, activation="relu", padding="same"))
model.add(BatchNormalization())
model.add(Conv2D(256, filter_size, activation="relu", padding="same"))
model.add(MaxPooling2D(pool_size=pool_size, strides=(2,2)))

model.add(Flatten())
model.add(Dense(2048, activation='relu'))
model.add(Dense(1024, activation='relu'))
model.add(Dense(output_size, activation='softmax'))
          
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=1e-4), metrics=['accuracy'])
model.summary()

Train Model

In [6]:
model.fit(X_train, y_train, epochs=10, batch_size=16, validation_data=(X_val, y_val))

Epoch 1/10
1471/1471 [==============================] - 348s 236ms/step - loss: 1.2528 - accuracy: 0.6303 - val_loss: 0.3138 - val_accuracy: 0.8975
Epoch 2/10
1471/1471 [==============================] - 356s 242ms/step - loss: 0.1674 - accuracy: 0.9470 - val_loss: 0.1062 - val_accuracy: 0.9689
Epoch 3/10
1471/1471 [==============================] - 357s 243ms/step - loss: 0.0815 - accuracy: 0.9744 - val_loss: 0.0850 - val_accuracy: 0.9769
Epoch 4/10
1471/1471 [==============================] - 353s 240ms/step - loss: 0.0575 - accuracy: 0.9824 - val_loss: 0.1240 - val_accuracy: 0.9626
Epoch 5/10
1471/1471 [==============================] - 360s 245ms/step - loss: 0.0480 - accuracy: 0.9856 - val_loss: 0.0765 - val_accuracy: 0.9818
Epoch 6/10
1471/1471 [==============================] - 358s 243ms/step - loss: 0.0401 - accuracy: 0.9881 - val_loss: 0.0492 - val_accuracy: 0.9862
Epoch 7/10
1471/1471 [==============================] - 358s 243ms/step - loss: 0.0299 - accuracy: 0.9914 - val_

Save model

In [7]:
model.save("traffic_sign_model.h5")

Load Model

In [8]:
model = keras.models.load_model("traffic_sign_model.h5")

Test Model

In [9]:
model.evaluate(X_test, y_test)

246/246 [==============================] - 17s 68ms/step - loss: 0.0740 - accuracy: 0.9850


[0.07396675646305084, 0.9849528074264526]